In [1]:
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('..')

In [2]:
from omegaconf import OmegaConf
import torch
import segmentation_models_pytorch as smp
from torch import nn
from pytorch_lightning.logging import TensorBoardLogger
from dl.lightning_model import *

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning:

can't resolve package from __spec__ or __package__, falling back on __name__ and __path__

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192

/usr/l

In [3]:
model = smp.FPN(encoder_name="resnext50_32x4d", classes=2, decoder_merge_policy='cat')
# class_weight = dataset["train"].numpixels[1,0:2]
# class_weight = torch.from_numpy(1-class_weight/class_weight.sum()).type('torch.FloatTensor')

criterion = nn.CrossEntropyLoss(
    ignore_index = -100,
    reduction = 'none'
)

In [4]:
conf = OmegaConf.create(
    {        
        'dataloader_args': {
            'batch_size':6,
            'n_classes':2
        },
        
        'optimizer_args':{
            'lr':0.001,
            'encoder_lr':0.0005,
            'weight_decay':0.0003,
            'encoder_weight_decay':0.00003
        },
        
        'scheduler_args': {
            'factor':0.25,
            'patience':2
        },
    
        'loss_args' : {
            'edge_weight' : 1.1,
        }

    }
)
print(conf.pretty())

dataloader_args:
  batch_size: 6
  n_classes: 2
optimizer_args:
  lr: 0.001
  encoder_lr: 0.0005
  weight_decay: 0.0003
  encoder_weight_decay: 3.0e-05
scheduler_args:
  factor: 0.25
  patience: 2
loss_args:
  edge_weight: 1.1



In [5]:
logdir = "../../../../results/tensorboard_logs/"
model_dir = "../../../../results/trained_models/"

In [6]:
%load_ext tensorboard
%tensorboard --logdir {logdir}

/usr/lib/python3.6/subprocess.py:786: ResourceWarning:

subprocess 17604 is still running



In [7]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath = model_dir,
    save_top_k = 1,
    save_last = True,
    verbose = True, 
    monitor = 'val_loss',
    mode = 'min',
    prefix = ''
)

logger = TensorBoardLogger(save_dir=logdir, name="FPN")

In [9]:
lightning_model = SegModel(model=model, criterion = criterion, hparams = conf, dataset='kumar')
trainer = pl.Trainer(
    default_root_dir=model_dir,
    max_epochs=30, 
    gpus=0, 
    checkpoint_callback=checkpoint_callback, 
    logger=logger
)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores


In [15]:
#trainer.logger.log_dir

'../../../../results/tensorboard_logs/FPN/version_0'

In [ ]:
trainer.fit(lightning_model)